# Structured Output Parser

- 著者: [Yoolim Han](https://github.com/hohosznta)
- ピアレビュー: [ranian963](https://github.com/ranian963), [asummerz](https://github.com/asummerz)
- 校正: [BokyungisaGod](https://github.com/BokyungisaGod)
- これは [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial) の一部です

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/03-StructuredOutputParser.ipynb)
[![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/03-StructuredOutputParser.ipynb)

## 概要

```StructuredOutputParser```は、大規模言語モデル(LLM)の応答を辞書構造にフォーマットし、複数のフィールドをキー/値のペアとして返すことができる貴重なツールです。
PydanticおよびJSONパーサーは堅牢な機能を提供しますが、```StructuredOutputParser```は、パラメータ数が少ないローカルモデルなど、性能が低いモデルに特に効果的です。GPTやClaudeのような高度なモデルと比較して知能が低いモデルには特に有益です。
```StructuredOutputParser```を活用することで、開発者はパラメータ数が少ないモデルを使用している場合でも、さまざまなLLMアプリケーション全体でデータの整合性と一貫性を維持できます。

### 目次

- [概要](#概要)
- [環境設定](#環境設定)
- [StructuredOutputParserの実装](#StructuredOutputParserの実装)

### 参考文献

- [LangChain ChatOpenAI APIリファレンス](https://python.langchain.com/docs/integrations/chat/openai/)
- [LangChain Structured output parser](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.structured.StructuredOutputParser.html#langchain.output_parsers.structured.StructuredOutputParser)
---

## 環境設定

環境をセットアップします。詳細については[環境設定](https://wikidocs.net/257836)を参照してください。

**[注意]**
- ```langchain-opentutorial```は、チュートリアル用の使いやすい環境設定と、便利な関数やユーティリティを提供するパッケージです。
- 詳細については[```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi)をご確認ください。

In [9]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [10]:
# 必要なパッケージをインストール
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
        "langchain_community",
    ],
    verbose=False,
    upgrade=False,
)

In [11]:
# 環境変数を設定
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "03-StructuredOutputParser",
    }
)

環境変数が正常に設定されました。


あるいは、```.env```ファイルに```OPENAI_API_KEY```を設定して読み込むこともできます。

[注意] 前の手順で```OPENAI_API_KEY```を既に設定している場合、これは不要です。

In [12]:
from dotenv import load_dotenv

load_dotenv(override=True)

False

## ```StructuredOutputParser```の実装

### ```ResponseSchema```と```StructuredOutputParser```の使用
*   ```ResponseSchema```クラスを使用して応答スキーマを定義し、ユーザーの質問への回答と、使用されたソース(ウェブサイト)の```description```を含めます。

*   ```response_schemas```を使用して```StructuredOutputParser```を初期化し、定義された応答スキーマに従って出力を構造化します。

**[注意]**
ローカルモデルを使用する場合、Pydanticパーサーが正常に動作しないことが頻繁にあります。そのような場合、```StructuredOutputParser```を使用することが良い代替解決策となります。

In [13]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# ユーザーの質問への応答
response_schemas = [
    ResponseSchema(name="answer", description="ユーザーの質問への回答"),
    ResponseSchema(
        name="source",
        description="ユーザーの質問に回答するために使用された`source`で、`ウェブサイトURL`である必要があります。",
    ),
]
# 応答スキーマに基づいて構造化出力パーサーを初期化
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

### プロンプトへの応答スキーマの埋め込み

```PromptTemplate```を作成して、ユーザーの質問をフォーマットし、構造化出力の解析指示を埋め込みます。

In [14]:
from langchain_core.prompts import PromptTemplate
# フォーマット指示を解析
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # ユーザーの質問にできるだけ適切に回答するテンプレートを設定
    template="ユーザーの質問にできるだけ適切に回答してください。\n{format_instructions}\n{question}",
    # 'question'を入力変数として使用
    input_variables=["question"],
    # 'format_instructions'を部分変数として使用
    partial_variables={"format_instructions": format_instructions},
)

### ```ChatOpenAI```との統合と```Chain```の実行

```PromptTemplate```、```ChatOpenAI```モデル、```StructuredOutputParser```を```chain```に結合します。最後に、特定の```question```で```chain```を実行して結果を生成します。

In [15]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)  # ChatOpenAIモデルを初期化

chain = prompt | model | output_parser  # プロンプト、モデル、出力パーサーを接続

# 「世界最大の砂漠は何ですか?」という質問をする
chain.invoke({"question": "世界最大の砂漠は何ですか?"})

{'answer': '世界最大の砂漠は南極砂漠です。',
 'source': 'https://www.worldatlas.com/articles/what-is-the-largest-desert-in-the-world.html'}

### ストリーミング出力の使用

```chain.stream```メソッドを使用して、「サッカーチームには何人のプレーヤーがいますか?」という```question```へのストリーミング応答を受け取ります。

In [16]:
for s in chain.stream({"question": "サッカーチームには何人のプレーヤーがいますか?"}):
    # 出力をストリーミング
    print(s)

{'answer': '標準的なサッカーチームは、フィールド上に一度に11人のプレーヤーがいます。', 'source': 'https://www.fifa.com/who-we-are/news/what-are-the-rules-of-football-2040008'}
